# Final Project, NSD Summer 2022

## Title: <font color=blue>An Unsupervised Deep Learning Approach to Solving Partial Integro-Differential Equations</font>

### Team member: Anyi Liu (刘安易),  Sitong Wei (魏斯桐), and Songqi Cao (曹淞琪)

$$
\newcommand{\supp}{\mathrm{supp}}
\newcommand{\E}{\mathbb{E} }
\newcommand{\Eof}[1]{\mathbb{E}\left[ #1 \right]}
\def\Cov{{ \mbox{Cov} }}
\def\Var{{ \mbox{Var} }}
\newcommand{\1}{\mathbf{1} }
\newcommand{\PP}{\mathbb{P} }
\newcommand{\Pof}[1]{\mathbb{P}\left[ #1 \right]}
%\newcommand{\Pr}{\mathrm{Pr} }
\newcommand{\QQ}{\mathbb{Q} }
\newcommand{\RR}{\mathbb{R} }
\newcommand{\DD}{\mathbb{D} }
\newcommand{\HH}{\mathbb{H} }
\newcommand{\spn}{\mathrm{span} }
\newcommand{\cov}{\mathrm{cov} }
\newcommand{\sgn}{\mathrm{sgn} }
\newcommand{\HS}{\mathcal{L}_{\mathrm{HS}} }
%\newcommand{\HS}{\mathrm{HS} }
\newcommand{\trace}{\mathrm{trace} }
\newcommand{\LL}{\mathcal{L} }
%\newcommand{\LL}{\mathrm{L} }
\newcommand{\s}{\mathcal{S} }
\newcommand{\ee}{\mathcal{E} }
\newcommand{\ff}{\mathcal{F} }
\newcommand{\hh}{\mathcal{H} }
\newcommand{\bb}{\mathcal{B} }
\newcommand{\dd}{\mathcal{D} }
\newcommand{\g}{\mathcal{G} }
\newcommand{\p}{\partial}
\newcommand{\half}{\frac{1}{2} }
\newcommand{\T}{\mathcal{T} }
\newcommand{\bi}{\begin{itemize}}
\newcommand{\ei}{\end{itemize}}
\newcommand{\beq}{\begin{equation}}
\newcommand{\eeq}{\end{equation}}
\newcommand{\beas}{\begin{eqnarray*}}
\newcommand{\eeas}{\end{eqnarray*}}
\newcommand{\cO}{\mathcal{O}}
\newcommand{\cF}{\mathcal{F}}
\newcommand{\cL}{\mathcal{L}}
\newcommand{\BS}{\text{BS}}
$$

## Abstract


In this Final Project, we reproduced the unsupervised deep learning approach to solving partial integro-differential equations reported in Weilong Fu & Ali Hirsa (2022) Quantitative Finance, 22:8, 1481-1494. Based on the underlying Lévy process with jumps, the partial integro-differential equation (PIDE) is employed for option pricing. The approach employs a neural network as the candidate solution and trains the neural network to satisfy the PIDE. It does not require any labels for training. By matching the PIDE and the boundary conditions, the neural network would yield an accurate solution to the PIDE. Additional singular terms are added to the neural network to satisfy the non-smooth initial conditions.

## Introduction


Lévy processes describes the fat tails of asset returns and matching theimplied volatility surfaces in option markets better than difusion models. The partial integro-differential equation (PIDE) is used to solve option prices under the models based on Lévy processes. The difference between a PIDE and a partial differential equation (PDE) is that a PIDE contains an integral term, which is generated by jumps in Lévy processes. For this reason, the PIDE is harder to be solved. Several mathematical approaches, such as the finite difference method or the fast Fourier transform, and approaches based on supervised deep learning have been utilized. 

In Weilong Fu & Ali Hirsa (2022) Quantitative Finance, 22:8, 1481-1494, an unsupervised deep learning approach to solving partial integro-differential equations has been reported. In the paper, apricing method of European/American calls/puts in the models based on Lévy processes by solving the PIDE with a neural network is proposed. The neural network is used as the approximated price surface. It only needs to be trained once and is then able to generate prices fast. The difference from supervised approaches is that this approach is self-contained, which does not need pre-calculated labels of option prices. In option pricing, the final payoff (the initial condition) is not a smooth function, which contradicts the smooth neural network. Thus, additional features called singular terms to satisfy the non-smooth property of the initial conditions is contained in the neural network. 

## I. Derivation of PIDEs from Lévy process

### 1. Lévy process
A stochastic process $\{X(t)\}_{t\geq0}$ is a Lévy process if it satisfies the following conditions:
* The process starts at zero almost surely;
* $X(t)$ has independent increments;
* $X(t)$ has stationary increments, that is, for every set of time instants $t_0<0<t_1<t_2<...<t_n$, the increments $X(t_1)-X(t_0)$, $X(t_2)-X(t_1)$, ..., $X(t_n)-X(t_{n-1})$ are identically distributed;
* It is continuous in probability, that is, for any given $\varepsilon>0$ and $t>0$, $\lim_{h\rightarrow0}\mathbb{P}[|X(t+h)-X(t)|>\varepsilon]=0$.

According to the Lévy–Khinchine theorem, a Lévy process iscompletely specified by its characteristic component:

$$
\psi(u)=-\frac{s^2}{2}u^2+\text{i}\gamma u+\int_{\mathbb{R}}(e^{\text{i}uy}-1-\text{i}uy·\mathbb{I}[|y|\leq1])m(\text{d}y)
$$

which satisfies $\mathbb{E}[e^{\text{i}uX(t)}]=e^{t\psi}$.Here $s\leq0$ and $\gamma$ are real constants and the Lévy measure $m(\text{d}y)$ is a positive measure on $\mathbb{R}$ that satisfies $\int_{\mathbb{R}}\min(1, y^2)m(\text{d}y)<\infty$.

If m satisfies $\int_{|y|>1}|y|m(\text{d}y)<\infty$, the characteristic component can be written as

$$
\psi(u)-\frac{s^2}{2}u^2+\text{i}\tilde{\gamma}u+\int_{\mathbb{R}}(e^{\text{i}uy}-1-\text{i}uy)m(\text{d}y)
$$

where $\tilde{\gamma}=\gamma+\int_{|y|>1}|y|m(\text{d}y)$.

### 2. Option pricing
The risk neutral stock price process for option pricing is defined as

$$
S(t)=S(0)\exp[(r-q)t+X(t)+\omega t]
$$

where $r$ is the risk-free interest rate, $q$ is the dividend rate of the stock, and $\omega$ is a constant such that $\mathbb{E}[S(t)]=S(0)e^{(r−q)t}$.

Suppose $\{S(t)\}_{t\geq0}$ is the stock price process, $K$ is the strike price, $t$ is the current time and $T$ is the maturity (expiration) time, the European/American call/put are defined as:

|  Option  | Definition |
|  :----: | :----: |
| European Call | $$c(S,t)=e^{-r(T-t)}\mathbb{E}[(S(T)-K)^+|S(t)=S]$$ |
| European Put | $$p(S,t)=e^{-r(T-t)}\mathbb{E}[(K-S(T))^+|S(t)=S]$$ |
| American Call | $$C(S,t)=\sup_{t\leq\bar{t}\leq T}\{e^{-r(\bar{t}-t)}\mathbb{E}[(S(\bar{t})-K^+|S(t)=S]\}$$ |
| American Put | $$P(S,t)=\sup_{t\leq\bar{t}\leq T}\{e^{-r(\bar{t}-t)}\mathbb{E}[(K-S(\bar{t}))^+|S(t)=S]\}$$ |

where $\bar{t}$ is an arbitrary stopping time between $t$ and $T$.

### 3. PIDE for option pricing
Let $x=\ln(S)$ be the log-price. According to the martingle approach, the PIDE is:

$$
\int_{-\infty}^\infty\Big(\tilde{w}(x+y,t)-\tilde{w}(x,t)-\frac{\partial\tilde{w}}{\partial x}(x,t)(e^y-1)\Big)m(\text{d}y)+\frac{\partial\tilde{w}}{\partial x}(x,t)+\frac{s^2}{2}\frac{\partial^2\tilde{w}}{\partial x^2}(x,t)+\Big(r-q-\frac{s^2}{2}\Big)\frac{\partial\tilde{w}}{\partial x}(x,t)-r\tilde{w}(x,t)=0
$$

Since the Lévy process is stationary, we look at the time to maturity $\tau = T − t$ instead of the calendar time $t$. Let $w(x,\tau)=\tilde{w}(x,t)$. thus the PIDE reads:

$$
H(w,x,\tau)=\int_{-\infty}^\infty\Big(w(x+y,\tau)-w(x,\tau)-\frac{\partial w}{\partial x}(x,\tau)(e^y-1)\Big)m(\text{d}y)-\frac{\partial w}{\partial x}(x,\tau)+\frac{s^2}{2}\frac{\partial^2w}{\partial x^2}(x,\tau)+\Big(r-q-\frac{s^2}{2}\Big)\frac{\partial w}{\partial x}(x,\tau)-rw(x,\tau)=0
$$

The European/American call/put can be solved by the equations:

|  Option  | Definition |
|  :----: | :----: |
| European Call |  $$H(w,x,\tau)=0$$ <br> $$w(x,0)=(e^x-K)^+$$  |
| European Put |  $$H(w,x,\tau)=0$$ <br> $$w(x,0)=(K-e^x)^+$$  |
| American Call |  $$\max[H(w,x,\tau),(e^x-K)^+-w(x,\tau)]$$ <br> $$w(x,0)=(e^x-K)^+$$  |
| American Put |  $$\max[H(w,x,\tau),(K-e^x)^+-w(x,\tau)]$$ <br> $$w(x,0)=(K-e^x)^+$$  |


## Neural network as the solution to the PIDE

###  1. Neural network solution
The solution $w(x,\tau)$ is not only a function of $x$ and $\tau$, but also depend on the parameters including $r$, $q$ and other model parameters of the Lévy process. The input of the neural network is $\boldsymbol{x}= (x,\tau,r,q;\theta_1,\theta_2, ... ,\theta_d)$ if there are $d$ model parameters. The output of the neural network is used as the approximation of $w(x,\tau)$. The neural network will be trained to satisfy the PIDEs.

### 2. Singular terms in the network
The solution $w(x,\tau)$ is not smooth at $(x,\tau)=(\ln(K),0)$. For example, European call prices of the Merton's model when $K=100$, $r=0$, $q=0$, $\sigma=0.1$, $\lambda=0.1$, $\alpha=0$ and $\delta=0.2$ has a singular point at $(x,\tau)=(ln(100),0) as shown below:


<img src="./figure_1.jpg" width="50%" ></img>


It causes a problem when we use the smooth neural network as the candidate solution, since the neural network is only able to fit a solution that is also smooth at$(x,\tau)=(\ln(K),0)$. A singular term is added in the neural network in order to circumvent this issue:

$$
\text{singular}(\boldsymbol{x})=\text{softplus}\Big(\frac{\text{main}(x,\tau,r,q)+\text{bias}(\boldsymbol{x})\tau}{\text{scale}(\boldsymbol{x})\sqrt{\tau}}\Big)·\text{scale}(\boldsymbol{x})\sqrt{\tau}
$$

where $\text{bias}(\boldsymbol{x})$ and $\text{scale}(\boldsymbol{x})$ are both neural networks, $\text{softplus}(z)=\ln(1+e^z)$, and $\text{main}(x,\tau,r,q)$ is defined as:

|  Option  | Definition |
|  :----: | :----: |
| European Call |  $$e^{x-q\tau}-Ke^{-r\tau}$$  |
| European Put |  $$Ke^{-r\tau}-e^{x-q\tau}$$  |
| American Call |  $$e^x-K$$  |
| American Put |  $$K-e^x$$  |

The formulism of this sigular term is inspired by Black-Scholes formula

\begin{array}
& c = N(d_1)Se^{-q\tau}-N(d_2)Ke^{-r\tau}\\
d_1 = \frac{\ln(S/K)+(r-q+\sigma^2/2)\tau)}{\sigma\sqrt{\tau}}\\
d_2 = \frac{\ln(S/K)+(r-q-\sigma^2/2)\tau)}{\sigma\sqrt{\tau}}
\end{array}

and it replicates the asymptotic behaviors of option price surfaces when $\tau\rightarrow0^+$ and $x\rightarrow\infty$.

However, asymmetric jumps in the Lévy process can generate option price surfaces much different from the shape of the singular term. If the stock price is driven by large jumps, the price curves are close to piecewise linear functions, and the linear parts are joined smoothly. For example, European call prices of the Merton's model when $K=100$, $r=0$, $q=0$, $\sigma=0.1$, $\lambda=1$, $\alpha=0.4$ and $\delta=0.1$, as illustrated below:


<img src="./figure_2.jpg" width="50%" ></img>


For this reason, a second singular term in the neural network is added to better model the curvature in the option surfaces under the
models based on Lévy processes. The second singular term is defined similarly to the first one as:
$$
\text{singular}_2(\boldsymbol{x})=\text{SiLU}\Big(\frac{\text{main}(x,\tau,r,q)+\text{bias}_2(\boldsymbol{x})\tau}{\text{scale}_2(\boldsymbol{x})\sqrt{\tau}}\Big)·\text{scale}_2(\boldsymbol{x})\sqrt{\tau}
$$

where $\text{SiLU}(z)=\frac{z}{1+e^{-z}}$.


### 3. Full structure of the neural network
The augmented MLP with the singular terms can be explained as follows:

\begin{array}
\boldsymbol{x}^{(0)}=\boldsymbol{x}\\
\boldsymbol{x}^{(j)}=g(\boldsymbol{W}^{(j-1)}\boldsymbol{x}^{(j-1)}+\boldsymbol{b}^{(j-1)}) &  1\leq j\leq L_1 \\
\text{bias}(\boldsymbol{x})=\boldsymbol{W}^{(\text{bias})}\boldsymbol{x}^{(L_1)}+\boldsymbol{b}^{(\text{bias})}\\
\text{scale}(\boldsymbol{x})=\text{softplus}(\boldsymbol{W}^{(\text{scale})}\boldsymbol{x}^{(L_1)}+\boldsymbol{b}^{(\text{scale})})\\
\text{bias}_2(\boldsymbol{x})=\boldsymbol{W}^{(\text{bias}_2)}\boldsymbol{x}^{(L_1)}+\boldsymbol{b}^{(\text{bias}_2)}\\
\text{scale}_2(\boldsymbol{x})=\text{softplus}(\boldsymbol{W}^{(\text{scale}_2)}\boldsymbol{x}^{(L_1)}+\boldsymbol{b}^{(\text{scale}_2)})\\
\text{singular}(\boldsymbol{x})=\text{softplus}\Big(\frac{\text{main}(x,\tau,r,q)+\text{bias}(\boldsymbol{x})\tau}{\text{scale}(\boldsymbol{x})\sqrt{\tau}}\Big)·\text{scale}(\boldsymbol{x})\sqrt{\tau}\\
\text{singular}_2(\boldsymbol{x})=\text{SiLU}\Big(\frac{\text{main}(x,\tau,r,q)+\text{bias}_2(\boldsymbol{x})\tau}{\text{scale}_2(\boldsymbol{x})\sqrt{\tau}}\Big)·\text{scale}_2(\boldsymbol{x})\sqrt{\tau}\\
\tilde{\boldsymbol{x}}^{(L_1)}=\text{concatenate}(x^{(L_1)},\text{singular}(\boldsymbol{x}),\text{singular}_2(\boldsymbol{x}))\\
\boldsymbol{x}^{(L_1+1)}=g(\boldsymbol{W}^{(L_1)}\tilde{\boldsymbol{x}}^{(L_1)}+\boldsymbol{b}^{(L_1)})\\
\boldsymbol{x}^{(j)}=g(\boldsymbol{W}^{(j-1)}\boldsymbol{x}^{(j-1)}+\boldsymbol{b}^{(j-1)}) &  L_1+1\leq j\leq L_1+L_2 \\
w=\sum_{0\leq j\leq L_1+L_2, j \neq  L_1}(\boldsymbol{W}^{(j,w)}\boldsymbol{x}^{(j)})+\boldsymbol{W}^{(L_1,w)}\tilde{\boldsymbol{x}}^{(L_1)}+b^{(w)}
\end{array}

The overall structure is of width $n$, with $L_1$ layers before thesingular terms and $L_2$ layers after the singular terms, as illustrated below


<img src="./figure_3.jpg" width="75%" ></img>


## Details in Neural Network

### 1. Initial and boundary conditions
For different kinds of options, the initial and boundary conditions are different. Let $x_{\min}$ ($x_{\max}$) be a small (large) enough constant, and $x$ will be sampled between $x_{\min}$ and $x_{\max}$ in the training process. The solution should satisfy the following conditions:

|  Option  | IC($x$) | $\text{BC}_{\min}(\tau,r,q)$ | $\text{BC}_{\max}(\tau,r,q)$ | 
|  :----: | :----: | :----: | :----: | 
| European Call |  $$(e^x-K)^+$$  |  $$0$$  |  $$e^{x_{\max}-q\tau}-Ke^{-r\tau}$$  |
| European Put |  $$(K-e^x)^+$$  |  $$Ke^{-r\tau}-e^{x_{\min}-q\tau}$$  |  $$0$$  |
| American Call |  $$(e^x-K)^+$$  |  $$0$$  |  $$e^{x_{\max}}-K$$  |
| American Put |  $$(K-e^x)^+$$  |  $$K-e^{x_{\min}}$$  |  $$0$$  |



### 2. Loss function
The parameters of the neural network are:

$$
\mathcal{W}=\{\boldsymbol{W}^{(j)},\boldsymbol{b}^{(j)}|0\leq j\leq L_1+L_2-1\}\bigcup\{\boldsymbol{W}^{(j,w)},b^{(w)}|0\leq j\leq L_1+L_2\}\bigcup\{\boldsymbol{W}^{(\text{bias})},\boldsymbol{b}^{(\text{bias})},\boldsymbol{W}^{(\text{bias}_2)},\boldsymbol{b}^{(\text{bias}_2)},\boldsymbol{W}^{(\text{scale})},\boldsymbol{b}^{(\text{scale})},\boldsymbol{W}^{(\text{scale}_2)},\boldsymbol{b}^{(\text{scale}_2)}\}
$$

The loss function of European puts and American puts are defined as follows:

$$
L_{\text{EU,P}}(\mathcal{W};\boldsymbol{x})=(H(w,x,\tau))^2+(w(x,0)-\text{IC}(x))^2+(w(x_{\min},\tau)-\text{BC}_{min}(\tau,r,q))^2+(w(x_{\max},\tau)-\text{BC}_{\max}(\tau,r,q))^2\\
L_{\text{AM,P}}(\mathcal{W};\boldsymbol{x})=(\max(H(w,x,\tau),\text{IC}(x)-w(x,\tau)))^2+(w(x,0)-\text{IC}(x))^2+(w(x_{\min},\tau)-\text{BC}_{min}(\tau,r,q))^2+(w(x_{\max},\tau)-\text{BC}_{\max}(\tau,r,q))^2
$$

this formulism of the loss function minimizes the difference on the intitial and boundary conditions.

However, for call options, $w(x,\tau)$ grows exponentially when $x$ is large. Samples with large $x$ may dominate in the gradient descent algorithm if the loss function has the same formulism with puts. In order to avoid the situation, and extra weight function is added:

$$
u(x)=\min(1,4K^2\exp(-2x))
$$

the loss functions of European calls and American calls are defined as follows:

$$
L_{\text{EU,C}}(\mathcal{W};\boldsymbol{x})=(H(w,x,\tau))^2u(x)+(w(x,0)-\text{IC}(x))^2+(w(x_{\min},\tau)-\text{BC}_{min}(\tau,r,q))^2+(w(x_{\max},\tau)-\text{BC}_{\max}(\tau,r,q))^2u(x_{\max})\\
L_{\text{AM,P}}(\mathcal{W};\boldsymbol{x})=(\max(H(w,x,\tau),\text{IC}(x)-w(x,\tau)))^2u(x)+(w(x,0)-\text{IC}(x))^2+(w(x_{\min},\tau)-\text{BC}_{min}(\tau,r,q))^2+(w(x_{\max},\tau)-\text{BC}_{\max}(\tau,r,q))^2u(x_{\max})
$$

The loss function above are defined on a single sample point. Given a set of sample points $\{\boldsymbol{x}_i|1\leq i\leq n\}$, the total loss function is an average of the individual ones:

$$
L_{\text{ave}}(\boldsymbol{W};\{\boldsymbol{x}_i\})=\frac1n\sum_iL(\boldsymbol{W};\boldsymbol{x}_i)
$$

### 3. Extrapolation of the price function
During the training process, the integral

$$
\int_{|y|>\varepsilon^+}(w(x+y,\tau)-w(x,\tau))m(\text{d}y)
$$

needs to be calculated. Here, $w(x,\tau)$ is approximated within the boundary $(x_{\min},x_{\max})$. However, $x+y$ could be out of the boundary if sample $x$ is close to the boundary. In this case, an extrapolation is needed. The extrapolation should be continuous at $x_{\min}$ and $x_{\max}$. The extrapolation for $w(x+y,\tau)-w(x,\tau)$ is:

| Option | $$x<x_{\min}$$ | $$x>x_{\max}$$ |
| :----: | :----: | :----: |
| European Call | 0 | $$(\exp(x)-\exp(x_{\max}))\exp(-q\tau)$$ |
| European Put | $$(\exp(x_{\min})-\exp(x))\exp(-q\tau)$$ | 0 |
| American Call | 0 | $$\exp(x)-\exp(x_{\max})$$ |
| American Put | $$\exp(x_{\min})-\exp(x)$$ | 0 |


### 4. Summarized algorithms

| **Training Route** |
| :---- |
| **Require**: A neural network $w(x,\tau)$ defined in part II.3, with parameters $\mathcal{W}$ |
| **for** iter = 1,2,3,...,$N$ **do** |
|     Generate a batch of random samples $\{\boldsymbol{x}_j\}^n$, where $n$ is the batch size |
|     Take one gradient descent step according to the loss function $L_{\text{ave}}$ |
| **end for** |

##  Codes

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
import os
import numpy as np
import scipy
import sobol_seq

import matplotlib.pyplot as plt
import time
import pickle
from tqdm import tqdm

from model.pricer import Pricer


data_dir = './'

In [ ]:
"""## Training routine"""
if __name__ == "__main__":

  ## create the pricer object
  obj = Pricer(model='VG', cp='call', exercise='American')
  save_name = '_'.join([obj.model,obj.exercise[:2],obj.cp])
  plot_paras = [{'T':1}]

  ## prepare all pre-calculation
  obj.data_preparer(train_size=500000,test_size=10000)

  ## build the network
  obj.net_builder(3, 3, 500)

  ## train the network
  opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
  obj.train(opt = opt, n_epochs=15, plot_paras = plot_paras)
  opt = tf.keras.optimizers.Adam(learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(1e-3, 15*(obj.train_size//200), 0.1))
  obj.train(opt = opt, n_epochs=15, plot_paras = plot_paras)
  opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
  obj.train(opt = opt, n_epochs=15, plot_paras = plot_paras)

  ## save the model
  obj.set_name(save_name)
  obj.save_model()

  ## evaluate the model over the test set
  pred = obj.predict()
  test_price = obj.test_price_fun()
  print(obj.compare(pred,test_price))

## Conclusion

## Reference (optional)
